In [1]:
# entropy, mutual information, kl divergence, cross entropy, conditional entropy
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Tuple, Dict


In [2]:
# entropy definition based on a value-probability dictionary
def entropy(p:Dict[str, float]) -> float:
    return -sum([p[i]*math.log2(p[i]) for i in p])

example_alphabet_probs={'a':0.5, 'b':0.25, 'c':0.25}
print(entropy(example_alphabet_probs))

1.5


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 858.3 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.5/79.5 MB 1.5 MB/s eta 0:00:44